# Functional / Reactive Programming in JavaScript

Five simple functions (some native to JavaScript and some included in the RxJS library):

* map
* filter
* concatAll
* reduce
* zip


# References

* [philosophical discussion](https://medium.com/javascript-scene/the-two-pillars-of-javascript-pt-2-functional-programming-a63aa53a41a4)
* [exercises](http://reactivex.io/learnrx/)
* [reactive programming](https://github.com/ReactiveX/rxjs)
* [highland.js - high-level streams](http://highlandjs.org/)

# Config

In [2]:
var newReleases = [
			{
				"id": 70111470,
				"title": "Die Hard",
				"boxart": "http://cdn-0.nflximg.com/images/2891/DieHard.jpg",
				"uri": "http://api.netflix.com/catalog/titles/movies/70111470",
				"rating": [4.0],
				"bookmark": []
			},
			{
				"id": 654356453,
				"title": "Bad Boys",
				"boxart": "http://cdn-0.nflximg.com/images/2891/BadBoys.jpg",
				"uri": "http://api.netflix.com/catalog/titles/movies/70111470",
				"rating": [5.0],
				"bookmark": [{ id: 432534, time: 65876586 }]
			},
			{
				"id": 65432445,
				"title": "The Chamber",
				"boxart": "http://cdn-0.nflximg.com/images/2891/TheChamber.jpg",
				"uri": "http://api.netflix.com/catalog/titles/movies/70111470",
				"rating": [4.0],
				"bookmark": []
			},
			{
				"id": 675465,
				"title": "Fracture",
				"boxart": "http://cdn-0.nflximg.com/images/2891/Fracture.jpg",
				"uri": "http://api.netflix.com/catalog/titles/movies/70111470",
				"rating": [5.0],
				"bookmark": [{ id: 432534, time: 65876586 }]
			}
		]

<br>
# Topics

<br>
__Traversing Arrays__

Unnecessarily verbose.  Don't need order.

In [4]:
var names = ["Ben", "Jafar", "Matt", "Priya", "Brian"]

var counter
for(counter = 0; counter < names.length; counter++) {
    console.log(names[counter]);
}

Ben
Jafar
Matt
Priya
Brian


<br>
Describes what we want to happen to each item in the array, but hides how the array is traversed

In [5]:
names.forEach( function(nm){console.log(nm)} )

Ben
Jafar
Matt
Priya
Brian


<br>
__Projecting Arrays__

All array projections share two operations in common:

* Traverse the source array
* Add each item's projected value to a new array

In [6]:
result = []
newReleases.forEach(function(nm){result.push({id:nm.id, title:nm.title})})
result

[ { id: 70111470, title: 'Die Hard' },
  { id: 654356453, title: 'Bad Boys' },
  { id: 65432445, title: 'The Chamber' },
  { id: 675465, title: 'Fracture' } ]

<br>
Abstract away how these operations are carried out

In [8]:
Array.prototype.map = function( projectionFun ){ 
    var result = []
    this.forEach( function(item){
        result.push( projectionFun(item) )
    })
    return result
}

[Function]

In [9]:
newReleases.map( function(item){return{id:item.id, title:item.title}})

[ { id: 70111470, title: 'Die Hard' },
  { id: 654356453, title: 'Bad Boys' },
  { id: 65432445, title: 'The Chamber' },
  { id: 675465, title: 'Fracture' } ]

<br>
__Filter__

In [12]:
videos = []
newReleases.forEach( function(item){if(item.rating >= 5.0){videos.push(item.title)}} )
videos

[ 'Bad Boys', 'Fracture' ]

The filter() function accepts a predicate. A predicate is a function that accepts an item in the array, and returns a boolean indicating whether the item should be retained in the new array.

In [13]:
Array.prototype.filter = function(predicateFunction) {
	var results = [];
	this.forEach(function(itemInArray) {
	  if (predicateFunction(itemInArray)) {
		results.push(itemInArray);
	  }
	});

	return results;
};

[Function]

Chaining together map() and filter() gives us a lot of expressive power. These high level functions let us express what data we want, but leave the underlying libraries a great deal of flexibility in terms of how our queries are executed.

In [23]:
newReleases.filter( 
    function(item){
        return item.rating >= 5.0
    } 
)

[ { id: 654356453,
    title: 'Bad Boys',
    boxart: 'http://cdn-0.nflximg.com/images/2891/BadBoys.jpg',
    uri: 'http://api.netflix.com/catalog/titles/movies/70111470',
    rating: [ 5 ],
    bookmark: [ [Object] ] },
  { id: 675465,
    title: 'Fracture',
    boxart: 'http://cdn-0.nflximg.com/images/2891/Fracture.jpg',
    uri: 'http://api.netflix.com/catalog/titles/movies/70111470',
    rating: [ 5 ],
    bookmark: [ [Object] ] } ]

In [22]:
newReleases.filter( 
    function(item){
        return item.rating >= 5.0
    } 
).map(
    function(item){
        return item.title
    }
)

[ 'Bad Boys', 'Fracture' ]

<br>
# Querying Trees

Trees pose a challenge because we need to flatten them into arrays in order to apply filter() and map() operations on them. In this section we'll define a concatAll() function that we can combine with map() and filter() to query trees.

In [46]:
var movieLists = [
			{
				name: "New Releases",
				videos: [
					{
						"id": 70111470,
						"title": "Die Hard",
						"boxart": "http://cdn-0.nflximg.com/images/2891/DieHard.jpg",
						"uri": "http://api.netflix.com/catalog/titles/movies/70111470",
						"rating": 4.0,
						"bookmark": []
					},
					{
						"id": 654356453,
						"title": "Bad Boys",
						"boxart": "http://cdn-0.nflximg.com/images/2891/BadBoys.jpg",
						"uri": "http://api.netflix.com/catalog/titles/movies/70111470",
						"rating": 5.0,
						"bookmark": [{ id: 432534, time: 65876586 }]
					}
				]
			},
			{
				name: "Dramas",
				videos: [
					{
						"id": 65432445,
						"title": "The Chamber",
						"boxart": "http://cdn-0.nflximg.com/images/2891/TheChamber.jpg",
						"uri": "http://api.netflix.com/catalog/titles/movies/70111470",
						"rating": 4.0,
						"bookmark": []
					},
					{
						"id": 675465,
						"title": "Fracture",
						"boxart": "http://cdn-0.nflximg.com/images/2891/Fracture.jpg",
						"uri": "http://api.netflix.com/catalog/titles/movies/70111470",
						"rating": 5.0,
						"bookmark": [{ id: 432534, time: 65876586 }]
					}
				]
			}
		]

In [42]:
var allVideoIdsInMovieLists = []

movieLists.forEach(function(movieList){
    movieList.videos.forEach(function(item){
        allVideoIdsInMovieLists.push(item.id) 
    })
})
allVideoIdsInMovieLists

[ 70111470, 654356453, 65432445, 675465 ]

Define a function that's abstract enough to express our intent to flatten a tree, without specifying too much information about how to carry out the operation.

In [48]:
Array.prototype.concatAll = function() {
	var results = [];
	this.forEach(function(subArray) {
		results.push.apply(results, subArray);
	});
	return results;
};

[Function]

In [49]:
JSON.stringify([ [1,2,3], [4,5,6], [7,8,9] ].concatAll()) === "[1,2,3,4,5,6,7,8,9]"

true

In [ ]:
[1,2,3].concatAll(); // throws an error because this is a one-dimensional array

# STOPPED AT EXERCISE 11

In [54]:
movieLists.
	  map(function(movieList) {
		return movieList.videos.map(function(video) {
			return video.id;
		  });
	  }).
	  concatAll();

TypeError: CreateListFromArrayLike called on non-object